La première étape de notre travail a été de récupérer les diverses données dont nous avions besoin afin d'essayer d'estimer l'impact des influenceurs de Youtube mais aussi des prix sur les livres les plus lus.

PARTIE 1: Récupérer des données sur les livres les plus lus, de différente manière

Récupération de l'API des bibliothèques de Paris afin d'estimer les cent livres les plus empruntés

Tout d'abord, formulons notre requête à l'API. On met comme limite 100 ouvrages car il s'agit de la limite de collecte possible au sein de l'API

In [49]:
import requests

response = requests.get("https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/les-1000-titres-les-plus-reserves-dans-les-bibliotheques-de-pret/records?limit=100")

# Afficher le type de contenu et le texte brut de la réponse
print("Type de contenu :", response.headers.get("Content-Type"))

Type de contenu : application/json; charset=utf-8


Une fois l'API récupérée, on observe la structure du jeu de données pour savoir comment s'appelle la variable 'résultat' ( ici 'results') à appeler afin de créer notre dataframe panda.
On observe que tous les rangs ne sont pas dans la base. Notre hypothèse optimiste est qu'il y a peut-être des types d'ouvrage non classés dans cette base ( et ne nous intéressant pas puisque nous nous intéressons seulement aux livres, dont la catégorie existe dans la base). C'est une hypothèse qui nous semble probable étant donné que nous avons par la suite créé nous-même des trous dans le classement en enlevant les DVDs de ce dernier. Cependant, nous ne pouvons pas en être sûrs.

In [50]:
wb_bibli = response.json()  # Utilisation de .json() sur l'objet réponse
print("Structure du JSON :")
print(wb_bibli)

Structure du JSON :
{'total_count': 1000, 'results': [{'rang': 12, 'type_de_document': 'Livre adulte', 'reservations': 522.0, 'titre': "Réinventer l'amour : comment le patriarcat sabote les relations hétérosexuelles", 'auteur': 'Chollet,  Mona'}, {'rang': 13, 'type_de_document': 'Bande dessinée adulte', 'reservations': 521.0, 'titre': 'Le jeune acteur', 'auteur': 'Sattouf,  Riad'}, {'rang': 14, 'type_de_document': 'Bande dessinée adulte', 'reservations': 510.0, 'titre': 'Une jeunesse au Moyen-Orient, 1992-1994', 'auteur': 'Sattouf,  Riad'}, {'rang': 37, 'type_de_document': 'DVD tous publics', 'reservations': 337.0, 'titre': 'Dune', 'auteur': None}, {'rang': 58, 'type_de_document': 'DVD tous publics', 'reservations': 266.0, 'titre': 'Onoda. 10 000 nuits dans la jungle', 'auteur': None}, {'rang': 73, 'type_de_document': 'Nouveauté', 'reservations': 241.0, 'titre': 'Chien 51 : roman', 'auteur': 'Gaudé,  Laurent'}, {'rang': 87, 'type_de_document': 'Livre adulte', 'reservations': 225.0, 'ti

Création du dataframe.
On remarque qu'il y a parfois des catégories qui ne donnent pas assez d'information sur le type d'ouvrage, à l'image de "Nouveauté", ou encore de "None". Nous pensons qu'il peut être pertinent d'étudier l'impact différé des prix et de l'exposition sur Youtube selon le genre d'ouvrage , dans la limite du possible. Ainsi, nous avons décidé d'utiliser des méthodes de scraping pour compléter la base. Avant de faire cela, nous avons décidé d'enlever de la base tous les ouvrages de type DVDs, ces derniers ne nous intéressant pas dans le cadre de notre projet.

In [51]:
import pandas as pd
df_bibli= pd.json_normalize(wb['results'])
df_bibli.head(12)

,rang,type_de_document,reservations,titre,auteur
0,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona"
1,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad"
2,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad"
3,37,DVD tous publics,337.0,Dune,None
4,58,DVD tous publics,266.0,Onoda. 10 000 nuits dans la jungle,None
5,73,Nouveauté,241.0,Chien 51 : roman,"Gaudé, Laurent"
6,87,Livre adulte,225.0,Le gosse : roman,"Olmi, Véronique"
7,109,Bande dessinée jeunesse,196.0,Attaque au clair de lune,"Oda, Eiichiro"
8,125,DVD nouveautés tous publics,177.0,West Side Story,None
9,136,None,170.0,Etés anglais,"Howard, Elizabeth Jane"


Tout d'abord, il est nécessaire de prendre connaissance de toutes les catégories pour savoir lesquelles remplacer, desquelles se débarasser...

In [52]:
print(df_bibli["type_de_document"].unique())

['Livre adulte' 'Bande dessinée adulte' 'DVD tous publics' 'Nouveauté'
 'Bande dessinée jeunesse' 'DVD nouveautés tous publics' None
 'Nouveauté jeunesse' 'Bande dessinée ado' 'Livre ado' 'Livre jeunesse']


Comme annoncé, on ne garde que les catégories qui nous intéresse ci-dessous.

In [53]:
doc = ["Livre adulte", "Bande dessinée adulte", "Nouveauté", "Bande dessinée jeunesse", "Nouveauté jeunesse",
 "Bande dessinée ado", "Livre ado", "Livre jeunesse", None]
df_bibli= df_bibli[df_bibli["type_de_document"].isin(doc)]

In [54]:
df_bibli.head(5)

,rang,type_de_document,reservations,titre,auteur
0,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona"
1,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad"
2,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad"
5,73,Nouveauté,241.0,Chien 51 : roman,"Gaudé, Laurent"
6,87,Livre adulte,225.0,Le gosse : roman,"Olmi, Véronique"


On voit qu'il y a beaucoup de livres dans notre base dont le type n'est pas renseigné.

In [55]:
nouveau = ["Nouveauté", "Nouveauté jeunesse", None]
df_bibli_nouveau =  df_bibli[df_bibli["type_de_document"].isin(nouveau)]
print(df_bibli_nouveau)

    rang    type_de_document  reservations  \
5     73           Nouveauté         241.0   
9    136                None         170.0   
16   360                None         100.0   
19   394  Nouveauté jeunesse          95.0   
20   399           Nouveauté          95.0   
22   425  Nouveauté jeunesse          91.0   
29   501                None          84.0   
34   672           Nouveauté          71.0   
39   718                None          69.0   
40   720  Nouveauté jeunesse          68.0   
45   840                None          62.0   
46   860                None          61.0   
49   910           Nouveauté          60.0   
51   945           Nouveauté          58.0   
52   949           Nouveauté          58.0   
60    61                None         261.0   
66   217           Nouveauté         132.0   
69   233           Nouveauté         127.0   
71   275           Nouveauté         114.0   
74   333                None         103.0   
83   572                None      

Pour le scrapping, nous avons choisi de le faire sur livraddict puisqu'il s'agit d'un site où tous les titres sont en français, avec un type de fonction recherche pratique pour scrapper. Sur ce site sont de plus décrits tous les livres connus avec des catégorie toujours indiquées avec un code similaire.

Pour ce faire, nous avons d'abord générer une URL de recherche sur livraddict. On scrappe la page pour récupérer l'URL du premier résultat de la recherche

In [56]:
import requests
from bs4 import BeautifulSoup

# Fonction pour rechercher un livre sur Babelio et accéder à la page du premier résultat
def search_livraddict(book_title):
    # URL de recherche sur Livraddict
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    search_url = f'https://www.livraddict.com/search.php?t={book_title}]'

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    # Trouver le premier élément correspondant au lien d'un résultat
    page = BeautifulSoup(response, "html.parser")
    first_result = page.select_one('.listing_recherche li .item_photo a')
    if first_result is not None:
        url = first_result['href']
        full_url = f"https://www.livraddict.com{url}"
        return full_url
    else:
        return None

On scrappe cette fois la page du premier résultat de la recherche précédente pour trouver les informations du livre.

In [57]:
def type_livre(info):
    if info is None:
        type_livre="Nom du livre pas trouvé sur Livraddict"
    else:
        # Normaliser les chaînes (supprimer les espaces et convertir en minuscules)
        info_0_normalized = info[0].strip().lower()
        info_1_normalized = info[1].strip().lower()

        if info_0_normalized in ["album", "artbook/beau livre", "bande-dessinée", "comics", "manga"] and info_1_normalized in ["petite enfance", "enfance"]:
            type_livre = "Bande dessinée jeunesse"

        elif info_0_normalized in ["album", "artbook/beau livre", "bande-dessinée", "comics", "manga"] and info_1_normalized in ["adolescence"]:
            type_livre = "Bande dessinée ado"

        elif info_0_normalized in ["album", "artbook/beau livre", "bande-dessinée", "comics", "manga"] and info_1_normalized in ["young adult", "adulte"]:
            type_livre = "Bande dessinée ado"

        elif info_0_normalized in ["correspondance", "documentaire", "essai", "livre pratique", "nouvelle(s)", "poésie", "roman", "théâtre"] and info_1_normalized in ["petite enfance", "enfance"]:
            type_livre = "Livre jeunesse"

        elif info_0_normalized in ["correspondance", "documentaire", "essai", "livre pratique", "nouvelle(s)", "poésie", "roman", "théâtre"] and info_1_normalized in ["adolescence"]:
            type_livre = "Livre ado"

        elif info_0_normalized in ["correspondance", "documentaire", "essai", "livre pratique", "nouvelle(s)", "poésie", "roman", "théâtre"] and info_1_normalized in ["young adult", "adulte"]:
            type_livre = "Livre adulte"

        else: type_livre= "Introuvable"
    return type_livre


In [36]:
def obtain_type(url):
    if url is not None:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers).content
        page = BeautifulSoup(response, "html.parser")
        c_format= page.find("div", class_= "book_classif book_format mr1")
        c_lectorat= page.find("div", class_="book_classif book_lectorat mr1")
        if c_format:
            a_format = c_format.find("a")
            format_livre = a_format.text
        else:
            format_livre = "Non renseigné"
        if c_lectorat:
            a_lectorat = c_lectorat.find("a")
            lectorat = a_lectorat.text
        else: 
            lectorat = "Non renseigné"
        info = [format_livre, lectorat]
        return info
    else:
        return None

Création d'une fonction qui assemble ces deux fonctions pour qu'à partir seulement d'un titre, on obtienne le type du livre.

In [37]:
def search_type_livre(titre):
    livre = titre.split()
    titre_livre = ""
    for i in range(0, len(livre) - 1):
        titre_livre = titre_livre + livre[i] + "+"
    titre_livre = titre_livre + livre[len(livre)-1]
    url = search_livraddict(titre_livre)
    infos = obtain_type(url)
    type_de_livre = type_livre(infos)
    return type_de_livre

Maintenant, on définit pour quel type de titre quelle fonction s'applique et pour quels livres on va chercher leur type.

In [38]:
def apply_search(row):
    nouveau = ["Nouveauté", "Nouveauté jeunesse", None]
    if row["type_de_document"] in nouveau:
        parties = row["titre"].split(".")
        parties_bis = row["titre"].split(":")

        #pour les séries avec nom de série, numéro tome, nom du tome
        if len(parties)==3:
            saga = parties[0].strip()
            tome = parties[1].strip()
            nom = parties[2].strip()
            if int(tome) < 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+"0"+tome)
                if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
                    type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
            elif int(tome)>= 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
        
        #pour les séries avec que nom de série et numéro de tome
        elif len(parties)==2:
            saga = parties[0].strip()
            tome = parties[1].strip()
            if int(tome) < 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+"0"+tome)
                if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
                    type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
            elif int(tome)>= 10:
                type_de_livre = search_type_livre(saga+"+"+"tome"+"+"+tome)
        
        #pour les livre où il y a écrit le genre après le titre
        elif len(parties_bis) == 2:
            type_de_livre = search_type_livre(row["titre"])
            if type_de_livre == "Nom du livre pas trouvé sur Livraddict":
                nom = parties_bis[0].strip()
                type_de_livre = search_type_livre(nom)

        else:
            type_de_livre = search_type_livre(row["titre"])
        return type_de_livre


    else:
        return row["type_de_document"]

On applique enfin la fonction à notre dataframe.

In [58]:
df_bibli['type_de_document'] = df_bibli.apply(apply_search, axis=1)

On regarde si des livres n'ont pas été trouvés. On remarque que tous les types de livre ont bien été trouvé ! Hourra

In [40]:
erreur_recherche = ["Nom du livre pas trouvé sur Livraddict"]
df_bibli_erreur =  df_bibli[df_bibli["type_de_document"].isin(erreur_recherche)]
print(df_bibli_erreur)

Empty DataFrame
Columns: [rang, type_de_document, reservations, titre, auteur]
Index: []


In [41]:
autre_erreur = ["Introuvable"]
df_bibli_int =  df_bibli[df_bibli["type_de_document"].isin(erreur_recherche)]
print(df_bibli_int)

Empty DataFrame
Columns: [rang, type_de_document, reservations, titre, auteur]
Index: []


Maintenant, on veut réordonner la base de manière à pouvoir mettre des rangs sans trous dans les numéros sur cette dernière. On commence pour ce faire par l'ordonner selon le rang, afin ensuite de numéroter les lignes dans l'ordre d'affichage.

In [60]:
df_bibli = df_bibli.sort_values(by="rang")
df_bibli = df_bibli.reset_index(drop=True)

In [61]:
nb_books = df_bibli[df_bibli['titre'].notna()].shape[0]
print(nb_books)

86


In [62]:
df_bibli['classement'] = None  # Initialiser avec None

# Remplir `classement` uniquement jusqu'à l'index 85
max_index = 85  # Exemple d'index limite
df_bibli.loc[df_bibli.index <= max_index, 'classement'] = range(1, min(len(df_bibli), max_index + 1) + 1)

On vérifie pour finir le début de notre base pour s'assurer que notre variable marche bien.

In [63]:
df_bibli.head(10)

,rang,type_de_document,reservations,titre,auteur,classement
0,5,Livre adulte,755.0,La décision : roman,"Tuil, Karine",1
1,12,Livre adulte,522.0,Réinventer l'amour : comment le patriarcat sab...,"Chollet, Mona",2
2,13,Bande dessinée adulte,521.0,Le jeune acteur,"Sattouf, Riad",3
3,14,Bande dessinée adulte,510.0,"Une jeunesse au Moyen-Orient, 1992-1994","Sattouf, Riad",4
4,25,Livre adulte,398.0,Dans les brumes de Capelans,"Norek, Olivier",5
5,28,Livre adulte,367.0,Ton absence n'est que ténèbres : roman,Jón Kalman Stefánsson,6
6,33,Livre adulte,345.0,Vivre avec nos morts : petit traité de consola...,"Horvilleur, Delphine",7
7,61,Livre adulte,261.0,Faites votre glucose révolution : la formule s...,"Inchauspé, Jessie",8
8,72,Livre adulte,246.0,Le pays des autres : roman,"Slimani, Leïla",9
9,73,Livre adulte,241.0,Chien 51 : roman,"Gaudé, Laurent",10
